Re-plot Figure 2 during different bin intervals. See Fig S3.

In [2]:
import pandas as pd 
import numpy as np
import os.path
import matplotlib.pyplot as plt
import seaborn as sns
import glob

%matplotlib inline
sns.set_style("white")

base_dir = os.path.dirname(os.getcwd())

plt.rcParams['font.sans-serif'] = "Arial"
plt.rcParams["font.family"] = "sans-serif"
sns.set_style(style='ticks')

In [6]:
bin_intervals = [500, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000]
bin_labels = [0.5, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
titles = ['0.5 seconds', '1 second', '2 seconds', '3 seconds', '4 seconds', 
            '5 seconds', '6 seconds', '7 seconds', '8 seconds', '9 seconds', '10 seconds']

for j in range(len(bin_intervals)):

    long_gap_connection = pd.DataFrame()
    long_gap_connection = long_gap_connection.fillna(0)
    counter = 0

    flist = glob.glob(os.path.join(base_dir, 'Analyses', 'turn_taking', 'strangers_pure', '*.csv'))

    for file in flist:

        id_1 = file.split('/')[-1].split('_')[0]
        id_2 = file.split('_')[-1].split('.csv')[0]

        data = pd.read_csv(file)

        long_gaps = data.loc[data['gap_length'] > 2000]

        if len(long_gaps) > 0:

            connection_1 = pd.read_csv(os.path.join(base_dir, 'Data', 'continuous_connection_ratings', 'strangers', '{}_{}.csv'.format(id_1, id_2)))
            connection_2 = pd.read_csv(os.path.join(base_dir, 'Data', 'continuous_connection_ratings', 'strangers', '{}_{}.csv'.format(id_2, id_1)))

            connection_1['time_msec'] = connection_1['adjustedTime'] * 1000
            connection_2['time_msec'] = connection_2['adjustedTime'] * 1000

            for i in list(long_gaps.index): 

                if (i > 0):

                    if (data.at[i-1, 'turn_end_msec'] > 6000) & (data.at[i, 'turn_start_msec'] < 594000): 

                        # define the long gap

                        start_of_gap = data.at[i-1, 'turn_end_msec']
                        end_of_gap = data.at[i, 'turn_start_msec']
                        
                        # 3 chunks before long gap

                        start_connection_before_3 = start_of_gap - (bin_intervals[j] * 3)
                        end_connection_before_3 = start_of_gap - (bin_intervals[j] * 2)

                        connection_1_before_3_subset = connection_1.loc[(connection_1['time_msec'] > start_connection_before_3) & (connection_1['time_msec'] < end_connection_before_3)].reset_index(drop=True)
                        connection_1_before_long_gap_3 = np.mean(connection_1_before_3_subset['Rating'])

                        connection_2_before_3_subset = connection_2.loc[(connection_2['time_msec'] > start_connection_before_3) & (connection_2['time_msec'] < end_connection_before_3)].reset_index(drop=True)
                        connection_2_before_long_gap_3 = np.mean(connection_2_before_3_subset['Rating'])

                        long_gap_connection.at[counter, 'subID'] = id_1
                        long_gap_connection.at[counter, 'partnerID'] = id_2
                        long_gap_connection.at[counter, 'dyad'] = id_1 + '_' + id_2
                        long_gap_connection.at[counter, 'timepoint'] = '3_before'
                        long_gap_connection.at[counter, 'connection'] = connection_1_before_long_gap_3
                        long_gap_connection.at[counter, 'connection_change'] = connection_1_before_long_gap_3 - connection_1_before_long_gap_3
                        long_gap_connection.at[counter, 'gap_length'] = data.at[i, 'gap_length']

                        counter += 1

                        long_gap_connection.at[counter, 'subID'] = id_2
                        long_gap_connection.at[counter, 'partnerID'] = id_1
                        long_gap_connection.at[counter, 'dyad'] = id_1 + '_' + id_2
                        long_gap_connection.at[counter, 'timepoint'] = '3_before'
                        long_gap_connection.at[counter, 'connection'] = connection_2_before_long_gap_3
                        long_gap_connection.at[counter, 'connection_change'] = connection_2_before_long_gap_3 - connection_2_before_long_gap_3
                        long_gap_connection.at[counter, 'gap_length'] = data.at[i, 'gap_length']

                        counter += 1
                        
                        # the long gap

                        connection_1_long_gap_subset = connection_1.loc[(connection_1['time_msec'] > start_of_gap) & (connection_1['time_msec'] < end_of_gap)].reset_index(drop=True)
                        connection_1_during_long_gap = np.mean(connection_1_long_gap_subset['Rating'])

                        connection_2_long_gap_subset = connection_2.loc[(connection_2['time_msec'] > start_of_gap) & (connection_2['time_msec'] < end_of_gap)].reset_index(drop=True)
                        connection_2_during_long_gap = np.mean(connection_2_long_gap_subset['Rating'])

                        long_gap_connection.at[counter, 'subID'] = id_1
                        long_gap_connection.at[counter, 'partnerID'] = id_2
                        long_gap_connection.at[counter, 'dyad'] = id_1 + '_' + id_2
                        long_gap_connection.at[counter, 'timepoint'] = 'long_gap'
                        long_gap_connection.at[counter, 'connection'] = connection_1_during_long_gap
                        long_gap_connection.at[counter, 'connection_change'] = connection_1_during_long_gap - connection_1_before_long_gap_3
                        long_gap_connection.at[counter, 'gap_length'] = data.at[i, 'gap_length']

                        counter += 1

                        long_gap_connection.at[counter, 'subID'] = id_2
                        long_gap_connection.at[counter, 'partnerID'] = id_1
                        long_gap_connection.at[counter, 'dyad'] = id_1 + '_' + id_2
                        long_gap_connection.at[counter, 'timepoint'] = 'long_gap'
                        long_gap_connection.at[counter, 'connection'] = connection_2_during_long_gap
                        long_gap_connection.at[counter, 'connection_change'] = connection_2_during_long_gap - connection_2_before_long_gap_3
                        long_gap_connection.at[counter, 'gap_length'] = data.at[i, 'gap_length']

                        counter += 1

                        # 1 chunk before long gap

                        start_connection_before_1 = start_of_gap - bin_intervals[j]
                        end_connection_before_1 = start_of_gap

                        connection_1_before_1_subset = connection_1.loc[(connection_1['time_msec'] > start_connection_before_1) & (connection_1['time_msec'] < end_connection_before_1)].reset_index(drop=True)
                        connection_1_before_long_gap_1 = np.mean(connection_1_before_1_subset['Rating'])

                        connection_2_before_1_subset = connection_2.loc[(connection_2['time_msec'] > start_connection_before_1) & (connection_2['time_msec'] < end_connection_before_1)].reset_index(drop=True)
                        connection_2_before_long_gap_1 = np.mean(connection_2_before_1_subset['Rating'])

                        long_gap_connection.at[counter, 'subID'] = id_1
                        long_gap_connection.at[counter, 'partnerID'] = id_2
                        long_gap_connection.at[counter, 'dyad'] = id_1 + '_' + id_2
                        long_gap_connection.at[counter, 'timepoint'] = '1_before'
                        long_gap_connection.at[counter, 'connection'] = connection_1_before_long_gap_1
                        long_gap_connection.at[counter, 'connection_change'] = connection_1_before_long_gap_1 - connection_1_before_long_gap_3
                        long_gap_connection.at[counter, 'gap_length'] = data.at[i, 'gap_length']

                        counter += 1

                        long_gap_connection.at[counter, 'subID'] = id_2
                        long_gap_connection.at[counter, 'partnerID'] = id_1
                        long_gap_connection.at[counter, 'dyad'] = id_1 + '_' + id_2
                        long_gap_connection.at[counter, 'timepoint'] = '1_before'
                        long_gap_connection.at[counter, 'connection'] = connection_2_before_long_gap_1
                        long_gap_connection.at[counter, 'connection_change'] = connection_2_before_long_gap_1 - connection_2_before_long_gap_3
                        long_gap_connection.at[counter, 'gap_length'] = data.at[i, 'gap_length']

                        counter += 1

                        # 2 chunks before long gap

                        start_connection_before_2 = start_of_gap - (bin_intervals[j] * 2) 
                        end_connection_before_2 = start_of_gap - bin_intervals[j]

                        connection_1_before_2_subset = connection_1.loc[(connection_1['time_msec'] > start_connection_before_2) & (connection_1['time_msec'] < end_connection_before_2)].reset_index(drop=True)
                        connection_1_before_long_gap_2 = np.mean(connection_1_before_2_subset['Rating'])

                        connection_2_before_2_subset = connection_2.loc[(connection_2['time_msec'] > start_connection_before_2) & (connection_2['time_msec'] < end_connection_before_2)].reset_index(drop=True)
                        connection_2_before_long_gap_2 = np.mean(connection_2_before_2_subset['Rating'])

                        long_gap_connection.at[counter, 'subID'] = id_1
                        long_gap_connection.at[counter, 'partnerID'] = id_2
                        long_gap_connection.at[counter, 'dyad'] = id_1 + '_' + id_2
                        long_gap_connection.at[counter, 'timepoint'] = '2_before'
                        long_gap_connection.at[counter, 'connection'] = connection_1_before_long_gap_2
                        long_gap_connection.at[counter, 'connection_change'] = connection_1_before_long_gap_2 - connection_1_before_long_gap_3
                        long_gap_connection.at[counter, 'gap_length'] = data.at[i, 'gap_length']

                        counter += 1

                        long_gap_connection.at[counter, 'subID'] = id_2
                        long_gap_connection.at[counter, 'partnerID'] = id_1
                        long_gap_connection.at[counter, 'dyad'] = id_1 + '_' + id_2
                        long_gap_connection.at[counter, 'timepoint'] = '2_before'
                        long_gap_connection.at[counter, 'connection'] = connection_2_before_long_gap_2
                        long_gap_connection.at[counter, 'connection_change'] = connection_2_before_long_gap_2 - connection_2_before_long_gap_3
                        long_gap_connection.at[counter, 'gap_length'] = data.at[i, 'gap_length']

                        counter += 1

                        # 1 chunk after long gap 

                        start_connection_after_1 = end_of_gap
                        end_connection_after_1 = end_of_gap + bin_intervals[j]

                        connection_1_after_1_subset = connection_1.loc[(connection_1['time_msec'] > start_connection_after_1) & (connection_1['time_msec'] < end_connection_after_1)].reset_index(drop=True)
                        connection_1_after_long_gap_1 = np.mean(connection_1_after_1_subset['Rating'])

                        connection_2_after_1_subset = connection_2.loc[(connection_2['time_msec'] > start_connection_after_1) & (connection_2['time_msec'] < end_connection_after_1)].reset_index(drop=True)
                        connection_2_after_long_gap_1 = np.mean(connection_2_after_1_subset['Rating'])

                        long_gap_connection.at[counter, 'subID'] = id_1
                        long_gap_connection.at[counter, 'partnerID'] = id_2
                        long_gap_connection.at[counter, 'dyad'] = id_1 + '_' + id_2
                        long_gap_connection.at[counter, 'timepoint'] = '1_after'
                        long_gap_connection.at[counter, 'connection'] = connection_1_after_long_gap_1
                        long_gap_connection.at[counter, 'connection_change'] = connection_1_after_long_gap_1 - connection_1_before_long_gap_3
                        long_gap_connection.at[counter, 'gap_length'] = data.at[i, 'gap_length']

                        counter += 1

                        long_gap_connection.at[counter, 'subID'] = id_2
                        long_gap_connection.at[counter, 'partnerID'] = id_1
                        long_gap_connection.at[counter, 'dyad'] = id_1 + '_' + id_2
                        long_gap_connection.at[counter, 'timepoint'] = '1_after'
                        long_gap_connection.at[counter, 'connection'] = connection_2_after_long_gap_1
                        long_gap_connection.at[counter, 'connection_change'] = connection_2_after_long_gap_1 - connection_2_before_long_gap_3
                        long_gap_connection.at[counter, 'gap_length'] = data.at[i, 'gap_length']


                        counter += 1

                        # 2 chunks after long gap 

                        start_connection_after_2 = end_of_gap + bin_intervals[j] 
                        end_connection_after_2 = end_of_gap + (bin_intervals[j] * 2)

                        connection_1_after_2_subset = connection_1.loc[(connection_1['time_msec'] > start_connection_after_2) & (connection_1['time_msec'] < end_connection_after_2)].reset_index(drop=True)
                        connection_1_after_long_gap_2 = np.mean(connection_1_after_2_subset['Rating'])

                        connection_2_after_2_subset = connection_2.loc[(connection_2['time_msec'] > start_connection_after_2) & (connection_2['time_msec'] < end_connection_after_2)].reset_index(drop=True)
                        connection_2_after_long_gap_2 = np.mean(connection_2_after_2_subset['Rating'])

                        long_gap_connection.at[counter, 'subID'] = id_1
                        long_gap_connection.at[counter, 'partnerID'] = id_2
                        long_gap_connection.at[counter, 'dyad'] = id_1 + '_' + id_2
                        long_gap_connection.at[counter, 'timepoint'] = '2_after'
                        long_gap_connection.at[counter, 'connection'] = connection_1_after_long_gap_2
                        long_gap_connection.at[counter, 'connection_change'] = connection_1_after_long_gap_2 - connection_1_before_long_gap_3
                        long_gap_connection.at[counter, 'gap_length'] = data.at[i, 'gap_length']

                        counter += 1

                        long_gap_connection.at[counter, 'subID'] = id_2
                        long_gap_connection.at[counter, 'partnerID'] = id_1
                        long_gap_connection.at[counter, 'dyad'] = id_1 + '_' + id_2
                        long_gap_connection.at[counter, 'timepoint'] = '2_after'
                        long_gap_connection.at[counter, 'connection'] = connection_2_after_long_gap_2
                        long_gap_connection.at[counter, 'connection_change'] = connection_2_after_long_gap_2 - connection_2_before_long_gap_3
                        long_gap_connection.at[counter, 'gap_length'] = data.at[i, 'gap_length']

                        counter += 1

                        # 3 chunks after long gap 

                        start_connection_after_3 = end_of_gap + (bin_intervals[j] * 2)
                        end_connection_after_3 = end_of_gap + (bin_intervals[j] * 3)

                        connection_1_after_3_subset = connection_1.loc[(connection_1['time_msec'] > start_connection_after_3) & (connection_1['time_msec'] < end_connection_after_3)].reset_index(drop=True)
                        connection_1_after_long_gap_3 = np.mean(connection_1_after_3_subset['Rating'])

                        connection_2_after_3_subset = connection_2.loc[(connection_2['time_msec'] > start_connection_after_3) & (connection_2['time_msec'] < end_connection_after_3)].reset_index(drop=True)
                        connection_2_after_long_gap_3 = np.mean(connection_2_after_3_subset['Rating'])

                        long_gap_connection.at[counter, 'subID'] = id_1
                        long_gap_connection.at[counter, 'partnerID'] = id_2
                        long_gap_connection.at[counter, 'dyad'] = id_1 + '_' + id_2
                        long_gap_connection.at[counter, 'timepoint'] = '3_after'
                        long_gap_connection.at[counter, 'connection'] = connection_1_after_long_gap_3
                        long_gap_connection.at[counter, 'connection_change'] = connection_1_after_long_gap_3 - connection_1_before_long_gap_3
                        long_gap_connection.at[counter, 'gap_length'] = data.at[i, 'gap_length']

                        counter += 1

                        long_gap_connection.at[counter, 'subID'] = id_2
                        long_gap_connection.at[counter, 'partnerID'] = id_1
                        long_gap_connection.at[counter, 'dyad'] = id_1 + '_' + id_2
                        long_gap_connection.at[counter, 'timepoint'] = '3_after'
                        long_gap_connection.at[counter, 'connection'] = connection_2_after_long_gap_3
                        long_gap_connection.at[counter, 'connection_change'] = connection_2_after_long_gap_3 - connection_2_before_long_gap_3
                        long_gap_connection.at[counter, 'gap_length'] = data.at[i, 'gap_length']

                        counter += 1


    long_gap_connection_friends = pd.DataFrame()
    long_gap_connection_friends = long_gap_connection_friends.fillna(0)
    counter = 0

    flist = glob.glob(os.path.join(base_dir, 'Analyses', 'turn_taking', 'friends', '*.csv'))

    for file in flist:

        id_1 = file.split('/')[-1].split('_')[0]
        id_2 = file.split('_')[-1].split('.csv')[0]

        data = pd.read_csv(file)

        long_gaps = data.loc[data['gap_length'] > 2000]

        if len(long_gaps) > 0:

            connection_1 = pd.read_csv(os.path.join(base_dir, 'Data', 'continuous_connection_ratings', 'friends', '{}_{}.csv'.format(id_1, id_2)))
            connection_2 = pd.read_csv(os.path.join(base_dir, 'Data', 'continuous_connection_ratings', 'friends', '{}_{}.csv'.format(id_2, id_1)))

            connection_1['time_msec'] = connection_1['adjustedTime'] * 1000
            connection_2['time_msec'] = connection_2['adjustedTime'] * 1000

            for i in list(long_gaps.index): 

                if (i > 0):

                    if (data.at[i-1, 'turn_end_msec'] > 6000) & (data.at[i, 'turn_start_msec'] < 594000): 

                        # define the long gap

                        start_of_gap = data.at[i-1, 'turn_end_msec']
                        end_of_gap = data.at[i, 'turn_start_msec']
                        
                        # 3 chunks before long gap

                        start_connection_before_3 = start_of_gap - (bin_intervals[j] * 3)
                        end_connection_before_3 = start_of_gap - (bin_intervals[j] * 2)

                        connection_1_before_3_subset = connection_1.loc[(connection_1['time_msec'] > start_connection_before_3) & (connection_1['time_msec'] < end_connection_before_3)].reset_index(drop=True)
                        connection_1_before_long_gap_3 = np.mean(connection_1_before_3_subset['Rating'])

                        connection_2_before_3_subset = connection_2.loc[(connection_2['time_msec'] > start_connection_before_3) & (connection_2['time_msec'] < end_connection_before_3)].reset_index(drop=True)
                        connection_2_before_long_gap_3 = np.mean(connection_2_before_3_subset['Rating'])

                        long_gap_connection_friends.at[counter, 'subID'] = id_1
                        long_gap_connection_friends.at[counter, 'partnerID'] = id_2
                        long_gap_connection_friends.at[counter, 'dyad'] = id_1 + '_' + id_2
                        long_gap_connection_friends.at[counter, 'timepoint'] = '3_before'
                        long_gap_connection_friends.at[counter, 'connection'] = connection_1_before_long_gap_3
                        long_gap_connection_friends.at[counter, 'connection_change'] = connection_1_before_long_gap_3 - connection_1_before_long_gap_3
                        long_gap_connection_friends.at[counter, 'gap_length'] = data.at[i, 'gap_length']


                        counter += 1

                        long_gap_connection_friends.at[counter, 'subID'] = id_2
                        long_gap_connection_friends.at[counter, 'partnerID'] = id_1
                        long_gap_connection_friends.at[counter, 'dyad'] = id_1 + '_' + id_2
                        long_gap_connection_friends.at[counter, 'timepoint'] = '3_before'
                        long_gap_connection_friends.at[counter, 'connection'] = connection_2_before_long_gap_3
                        long_gap_connection_friends.at[counter, 'connection_change'] = connection_2_before_long_gap_3 - connection_2_before_long_gap_3
                        long_gap_connection_friends.at[counter, 'gap_length'] = data.at[i, 'gap_length']


                        counter += 1
                        
                        # the long gap

                        connection_1_long_gap_subset = connection_1.loc[(connection_1['time_msec'] > start_of_gap) & (connection_1['time_msec'] < end_of_gap)].reset_index(drop=True)
                        connection_1_during_long_gap = np.mean(connection_1_long_gap_subset['Rating'])

                        connection_2_long_gap_subset = connection_2.loc[(connection_2['time_msec'] > start_of_gap) & (connection_2['time_msec'] < end_of_gap)].reset_index(drop=True)
                        connection_2_during_long_gap = np.mean(connection_2_long_gap_subset['Rating'])

                        long_gap_connection_friends.at[counter, 'subID'] = id_1
                        long_gap_connection_friends.at[counter, 'partnerID'] = id_2
                        long_gap_connection_friends.at[counter, 'dyad'] = id_1 + '_' + id_2
                        long_gap_connection_friends.at[counter, 'timepoint'] = 'long_gap'
                        long_gap_connection_friends.at[counter, 'connection'] = connection_1_during_long_gap
                        long_gap_connection_friends.at[counter, 'connection_change'] = connection_1_during_long_gap - connection_1_before_long_gap_3
                        long_gap_connection_friends.at[counter, 'gap_length'] = data.at[i, 'gap_length']

                        counter += 1

                        long_gap_connection_friends.at[counter, 'subID'] = id_2
                        long_gap_connection_friends.at[counter, 'partnerID'] = id_1
                        long_gap_connection_friends.at[counter, 'dyad'] = id_1 + '_' + id_2
                        long_gap_connection_friends.at[counter, 'timepoint'] = 'long_gap'
                        long_gap_connection_friends.at[counter, 'connection'] = connection_2_during_long_gap
                        long_gap_connection_friends.at[counter, 'connection_change'] = connection_2_during_long_gap - connection_2_before_long_gap_3
                        long_gap_connection_friends.at[counter, 'gap_length'] = data.at[i, 'gap_length']

                        counter += 1

                        # 1 chunk before long gap

                        start_connection_before_1 = start_of_gap - bin_intervals[j]
                        end_connection_before_1 = start_of_gap

                        connection_1_before_1_subset = connection_1.loc[(connection_1['time_msec'] > start_connection_before_1) & (connection_1['time_msec'] < end_connection_before_1)].reset_index(drop=True)
                        connection_1_before_long_gap_1 = np.mean(connection_1_before_1_subset['Rating'])

                        connection_2_before_1_subset = connection_2.loc[(connection_2['time_msec'] > start_connection_before_1) & (connection_2['time_msec'] < end_connection_before_1)].reset_index(drop=True)
                        connection_2_before_long_gap_1 = np.mean(connection_2_before_1_subset['Rating'])

                        long_gap_connection_friends.at[counter, 'subID'] = id_1
                        long_gap_connection_friends.at[counter, 'partnerID'] = id_2
                        long_gap_connection_friends.at[counter, 'dyad'] = id_1 + '_' + id_2
                        long_gap_connection_friends.at[counter, 'timepoint'] = '1_before'
                        long_gap_connection_friends.at[counter, 'connection'] = connection_1_before_long_gap_1
                        long_gap_connection_friends.at[counter, 'connection_change'] = connection_1_before_long_gap_1 - connection_1_before_long_gap_3
                        long_gap_connection_friends.at[counter, 'gap_length'] = data.at[i, 'gap_length']


                        counter += 1

                        long_gap_connection_friends.at[counter, 'subID'] = id_2
                        long_gap_connection_friends.at[counter, 'partnerID'] = id_1
                        long_gap_connection_friends.at[counter, 'dyad'] = id_1 + '_' + id_2
                        long_gap_connection_friends.at[counter, 'timepoint'] = '1_before'
                        long_gap_connection_friends.at[counter, 'connection'] = connection_2_before_long_gap_1
                        long_gap_connection_friends.at[counter, 'connection_change'] = connection_2_before_long_gap_1 - connection_2_before_long_gap_3
                        long_gap_connection_friends.at[counter, 'gap_length'] = data.at[i, 'gap_length']


                        counter += 1

                        # 2 chunks before long gap

                        start_connection_before_2 = start_of_gap - (bin_intervals[j] * 2) 
                        end_connection_before_2 = start_of_gap - bin_intervals[j]

                        connection_1_before_2_subset = connection_1.loc[(connection_1['time_msec'] > start_connection_before_2) & (connection_1['time_msec'] < end_connection_before_2)].reset_index(drop=True)
                        connection_1_before_long_gap_2 = np.mean(connection_1_before_2_subset['Rating'])

                        connection_2_before_2_subset = connection_2.loc[(connection_2['time_msec'] > start_connection_before_2) & (connection_2['time_msec'] < end_connection_before_2)].reset_index(drop=True)
                        connection_2_before_long_gap_2 = np.mean(connection_2_before_2_subset['Rating'])

                        long_gap_connection_friends.at[counter, 'subID'] = id_1
                        long_gap_connection_friends.at[counter, 'partnerID'] = id_2
                        long_gap_connection_friends.at[counter, 'dyad'] = id_1 + '_' + id_2
                        long_gap_connection_friends.at[counter, 'timepoint'] = '2_before'
                        long_gap_connection_friends.at[counter, 'connection'] = connection_1_before_long_gap_2
                        long_gap_connection_friends.at[counter, 'connection_change'] = connection_1_before_long_gap_2 - connection_1_before_long_gap_3
                        long_gap_connection_friends.at[counter, 'gap_length'] = data.at[i, 'gap_length']


                        counter += 1

                        long_gap_connection_friends.at[counter, 'subID'] = id_2
                        long_gap_connection_friends.at[counter, 'partnerID'] = id_1
                        long_gap_connection_friends.at[counter, 'dyad'] = id_1 + '_' + id_2
                        long_gap_connection_friends.at[counter, 'timepoint'] = '2_before'
                        long_gap_connection_friends.at[counter, 'connection'] = connection_2_before_long_gap_2
                        long_gap_connection_friends.at[counter, 'connection_change'] = connection_2_before_long_gap_2 - connection_2_before_long_gap_3
                        long_gap_connection_friends.at[counter, 'gap_length'] = data.at[i, 'gap_length']


                        counter += 1


                        # 1 chunk after long gap 

                        start_connection_after_1 = end_of_gap
                        end_connection_after_1 = end_of_gap + bin_intervals[j]

                        connection_1_after_1_subset = connection_1.loc[(connection_1['time_msec'] > start_connection_after_1) & (connection_1['time_msec'] < end_connection_after_1)].reset_index(drop=True)
                        connection_1_after_long_gap_1 = np.mean(connection_1_after_1_subset['Rating'])

                        connection_2_after_1_subset = connection_2.loc[(connection_2['time_msec'] > start_connection_after_1) & (connection_2['time_msec'] < end_connection_after_1)].reset_index(drop=True)
                        connection_2_after_long_gap_1 = np.mean(connection_2_after_1_subset['Rating'])

                        long_gap_connection_friends.at[counter, 'subID'] = id_1
                        long_gap_connection_friends.at[counter, 'partnerID'] = id_2
                        long_gap_connection_friends.at[counter, 'dyad'] = id_1 + '_' + id_2
                        long_gap_connection_friends.at[counter, 'timepoint'] = '1_after'
                        long_gap_connection_friends.at[counter, 'connection'] = connection_1_after_long_gap_1
                        long_gap_connection_friends.at[counter, 'connection_change'] = connection_1_after_long_gap_1 - connection_1_before_long_gap_3
                        long_gap_connection_friends.at[counter, 'gap_length'] = data.at[i, 'gap_length']


                        counter += 1

                        long_gap_connection_friends.at[counter, 'subID'] = id_2
                        long_gap_connection_friends.at[counter, 'partnerID'] = id_1
                        long_gap_connection_friends.at[counter, 'dyad'] = id_1 + '_' + id_2
                        long_gap_connection_friends.at[counter, 'timepoint'] = '1_after'
                        long_gap_connection_friends.at[counter, 'connection'] = connection_2_after_long_gap_1
                        long_gap_connection_friends.at[counter, 'connection_change'] = connection_2_after_long_gap_1 - connection_2_before_long_gap_3
                        long_gap_connection_friends.at[counter, 'gap_length'] = data.at[i, 'gap_length']


                        counter += 1

                        # 2 chunks after long gap 

                        start_connection_after_2 = end_of_gap + bin_intervals[j] 
                        end_connection_after_2 = end_of_gap + (bin_intervals[j] * 2)

                        connection_1_after_2_subset = connection_1.loc[(connection_1['time_msec'] > start_connection_after_2) & (connection_1['time_msec'] < end_connection_after_2)].reset_index(drop=True)
                        connection_1_after_long_gap_2 = np.mean(connection_1_after_2_subset['Rating'])

                        connection_2_after_2_subset = connection_2.loc[(connection_2['time_msec'] > start_connection_after_2) & (connection_2['time_msec'] < end_connection_after_2)].reset_index(drop=True)
                        connection_2_after_long_gap_2 = np.mean(connection_2_after_2_subset['Rating'])

                        long_gap_connection_friends.at[counter, 'subID'] = id_1
                        long_gap_connection_friends.at[counter, 'partnerID'] = id_2
                        long_gap_connection_friends.at[counter, 'dyad'] = id_1 + '_' + id_2
                        long_gap_connection_friends.at[counter, 'timepoint'] = '2_after'
                        long_gap_connection_friends.at[counter, 'connection'] = connection_1_after_long_gap_2
                        long_gap_connection_friends.at[counter, 'connection_change'] = connection_1_after_long_gap_2 - connection_1_before_long_gap_3
                        long_gap_connection_friends.at[counter, 'gap_length'] = data.at[i, 'gap_length']


                        counter += 1

                        long_gap_connection_friends.at[counter, 'subID'] = id_2
                        long_gap_connection_friends.at[counter, 'partnerID'] = id_1
                        long_gap_connection_friends.at[counter, 'dyad'] = id_1 + '_' + id_2
                        long_gap_connection_friends.at[counter, 'timepoint'] = '2_after'
                        long_gap_connection_friends.at[counter, 'connection'] = connection_2_after_long_gap_2
                        long_gap_connection_friends.at[counter, 'connection_change'] = connection_2_after_long_gap_2 - connection_2_before_long_gap_3
                        long_gap_connection_friends.at[counter, 'gap_length'] = data.at[i, 'gap_length']


                        counter += 1

                        # 3 chunks after long gap 

                        start_connection_after_3 = end_of_gap + (bin_intervals[j] * 2)
                        end_connection_after_3 = end_of_gap + (bin_intervals[j] * 3)

                        connection_1_after_3_subset = connection_1.loc[(connection_1['time_msec'] > start_connection_after_3) & (connection_1['time_msec'] < end_connection_after_3)].reset_index(drop=True)
                        connection_1_after_long_gap_3 = np.mean(connection_1_after_3_subset['Rating'])

                        connection_2_after_3_subset = connection_2.loc[(connection_2['time_msec'] > start_connection_after_3) & (connection_2['time_msec'] < end_connection_after_3)].reset_index(drop=True)
                        connection_2_after_long_gap_3 = np.mean(connection_2_after_3_subset['Rating'])

                        long_gap_connection_friends.at[counter, 'subID'] = id_1
                        long_gap_connection_friends.at[counter, 'partnerID'] = id_2
                        long_gap_connection_friends.at[counter, 'dyad'] = id_1 + '_' + id_2
                        long_gap_connection_friends.at[counter, 'timepoint'] = '3_after'
                        long_gap_connection_friends.at[counter, 'connection'] = connection_1_after_long_gap_3
                        long_gap_connection_friends.at[counter, 'connection_change'] = connection_1_after_long_gap_3 - connection_1_before_long_gap_3
                        long_gap_connection_friends.at[counter, 'gap_length'] = data.at[i, 'gap_length']


                        counter += 1

                        long_gap_connection_friends.at[counter, 'subID'] = id_2
                        long_gap_connection_friends.at[counter, 'partnerID'] = id_1
                        long_gap_connection_friends.at[counter, 'dyad'] = id_1 + '_' + id_2
                        long_gap_connection_friends.at[counter, 'timepoint'] = '3_after'
                        long_gap_connection_friends.at[counter, 'connection'] = connection_2_after_long_gap_3
                        long_gap_connection_friends.at[counter, 'connection_change'] = connection_2_after_long_gap_3 - connection_2_before_long_gap_3
                        long_gap_connection_friends.at[counter, 'gap_length'] = data.at[i, 'gap_length']


                        counter += 1

    long_gap_connection['condition'] = 'strangers'
    long_gap_connection_friends['condition'] = 'friends'

    long_gaps_all = pd.merge(long_gap_connection, long_gap_connection_friends, how='outer')

    plt.figure(figsize=(8,5))

    timepoint_labels = ['{} seconds\nbefore\n(baseline)'.format(bin_labels[j]*3), 
                        '{} seconds\nbefore'.format(bin_labels[j]*2), 
                        '{} seconds\nbefore'.format(bin_labels[j]), 
                        'long gap',
                       '{} seconds\nafter'.format(bin_labels[j]), 
                        '{} seconds\nafter'.format(bin_labels[j]*2), 
                        '{} seconds\nafter'.format(bin_labels[j]*3)]

    friend_color = '#F46036'
    stranger_color = '#0FA3B1'

    ax = sns.pointplot(x="timepoint", y="connection_change", hue="condition",
                   data=long_gaps_all, dodge=False,
                  order=["3_before", "2_before", "1_before", 'long_gap', '1_after', '2_after', '3_after'],
                  palette = [stranger_color,friend_color], markers='o', ci=None)

    plt.xlabel("timepoint", fontsize=18, labelpad=20)
    plt.ylabel("connection rating", fontsize=18, labelpad=10)
    plt.title("bin interval = {}".format(titles[j]), fontsize=18)

    ax.tick_params(labelsize=13)
    ax.set(ylim=(-1.5, 4)) # CHANGE LATER
    plt.legend(fontsize='13')

    ax.set_xticklabels(timepoint_labels)

    ax.axvspan(2.75, 3.25, alpha=0.3, color='silver')

    sns.despine()
    plt.tight_layout()

    plt.savefig(os.path.join(base_dir, 'Figures', 'Supplement_different_intervals', '{}.png'.format(titles[j])), dpi=300) 
    plt.close()

